# Recreating Abhishek's 89.3% Start Kernel with fastai


I saw Abhishek's [notebook](https://www.kaggle.com/abhishek/tez-faster-and-easier-training-for-leaf-detection?scriptVersionId=47408263) a number of days ago and wanted to try and recreate it in a 1:1 fashion with fastai to ensure that it could perform similar.

This notebook will be the result of that process, and by the end of it we will have trained an equivalent model that can achieve 89.4% on the public LB.

## A few key differences between his and what other fastai-kernels are doing

There are a few pieces that differ in most other fastai kernels that don't quite breach the 89% threshold with a single model. I believe it comes down to two factors:

1. Data Augmentation
2. How they are performing TTA
3. The scheduling being used

**Data Augmentation**

In his notebook there are a few transforms that fastai does not have equivalents of, such as `HueSaturationValue`. We have `Hue` and `Saturation`, and `Value` has not been implemented yet (and I don't know quite enough yet about those transforms to work out the logic). 

**TTA**

In most of the fastai kernels I've seen, they follow a standard TTA regiment. Abhishek performs TTA 15 times, which I saw had a significant boost in accuracy compared to the standard 3+1 that fastai will do

**Scheduling**

More PyTorch related kernels are now fitting with a `CosineAnnealingWarmRestarts` scheduler, but fastai doesn't have an equivalent, right?

It actually does. So `WarmRestarts` `T_0` value simply dictates how long we should be calling a `fit` for, and as we will see later the proper scheduler to use is `fit_flat_cos` with a `start_pct` of 0 (thanks to @tanlikesmath for pointing this out to me!)

## Installing what we need

We'll be installing the `efficientnet-pytorch` library:

In [ ]:
!pip install efficientnet-pytorch -qqq

## Importing from the libraries

Next we'll want to bring in everything we want.

In [ ]:
from fastai.vision.all import *


import albumentations # Data Augmentation
from efficientnet_pytorch import EfficientNet # The Model


## Setting up our transforms

Now we need to use `albumentations` for everything, as that's what he does!

So, how can we manage this?

fastai has a helpful [tutorial](https://docs.fast.ai/tutorial.albumentations.html) on how to get started with albumentations, and we'll take it a step further.

First here is what we currently have:


In [ ]:
class AlbumentationsTransform(DisplayedTransform):
    def __init__(self, aug): self.aug = aug
    def encodes(self, img: PILImage):
        aug_img = self.aug(image=np.array(img))['image']
        return PILImage.create(aug_img)

While this transform *does* work, it assumes that everything is applied to both the training and the validation set, and doesn't let us put in custom pipelines for both. Let's change that:

In [ ]:
class AlbumentationsTransform(RandTransform):
    "A transform handler for multiple `Albumentation` transforms"
    split_idx,order=None,2
    def __init__(self, train_aug, valid_aug): store_attr()
    
    def before_call(self, b, split_idx):
        self.idx = split_idx
    
    def encodes(self, img: PILImage):
        if self.idx == 0:
            aug_img = self.train_aug(image=np.array(img))['image']
        else:
            aug_img = self.valid_aug(image=np.array(img))['image']
        return PILImage.create(aug_img)

What we have done here is allow for a `train_aug` and `valid_aug` transform pipeline (these can be `albumentation.Compose`'d transforms, as we will see later) and we've ensured that some transforms will only occur on the training set (when split_idx is 0) and on the validation set (1). Let's now recreate his data augmentation:

In [ ]:
def get_train_aug(): return albumentations.Compose([
            albumentations.RandomResizedCrop(256,256),
            albumentations.Transpose(p=0.5),
            albumentations.HorizontalFlip(p=0.5),
            albumentations.VerticalFlip(p=0.5),
            albumentations.ShiftScaleRotate(p=0.5),
            albumentations.HueSaturationValue(
                hue_shift_limit=0.2, 
                sat_shift_limit=0.2, 
                val_shift_limit=0.2, 
                p=0.5
            ),
            albumentations.RandomBrightnessContrast(
                brightness_limit=(-0.1,0.1), 
                contrast_limit=(-0.1, 0.1), 
                p=0.5
            ),
            albumentations.CoarseDropout(p=0.5),
            albumentations.Cutout(p=0.5)
])

In [ ]:
def get_valid_aug(): return albumentations.Compose([
    albumentations.CenterCrop(256,256, p=1.),
    albumentations.Resize(256,256)
], p=1.)

Since albumentations are operating on `PILImage`'s, these need to be in the item transforms
> Warning: do not include `Normalize` in here! Our transform expects to be able to generate a PILImage at the end, and adjusting the datatypes will cause issues!

In [ ]:
item_tfms = AlbumentationsTransform(get_train_aug(), get_valid_aug())

## Building our `DataBlock`

We now have everything setup to build our data! We'll follow my similar notebook's data approach from [here](https://www.kaggle.com/muellerzr/cassava-fastai-starter)

In [ ]:
set_seed(999)

In [ ]:
path = Path("../input")
data_path = path/'cassava-leaf-disease-classification'
df = pd.read_csv(data_path/'train.csv')
df['image_id'] = df['image_id'].apply(lambda x: f'train_images/{x}')

In [ ]:
df.head()

In [ ]:
idx2lbl = {0:"Cassava Bacterial Blight (CBB)",
          1:"Cassava Brown Streak Disease (CBSD)",
          2:"Cassava Green Mottle (CGM)",
          3:"Cassava Mosaic Disease (CMD)",
          4:"Healthy"}

df['label'].replace(idx2lbl, inplace=True)

In [ ]:
df.head()

In [ ]:
blocks = (ImageBlock, CategoryBlock)
splitter = RandomSplitter(valid_pct=0.2, seed=999)
def get_x(row): return data_path/row['image_id']
def get_y(row): return row['label']

In [ ]:
block = DataBlock(blocks=blocks,
                 get_x=get_x,
                 get_y=get_y,
                 splitter=splitter,
                 item_tfms=item_tfms,
                 batch_tfms=[Normalize.from_stats(*imagenet_stats)])

And build the `DataLoaders`:

In [ ]:
dls = block.dataloaders(df, bs=32, val_bs=64)

I can now safely say we have *equivalent* data to Abhishek's. Let's look at a batch:

In [ ]:
dls.show_batch(figsize=(12,12))

## Building the Model

Next we'll want to build our model. I've recreated it in raw PyTorch below:

In [ ]:
class LeafModel(Module):
    def __init__(self, num_classes):

        self.effnet = EfficientNet.from_pretrained("efficientnet-b3")
        self.dropout = nn.Dropout(0.1)
        self.out = nn.Linear(1536, num_classes)

    def forward(self, image):
        batch_size, _, _, _ = image.shape

        x = self.effnet.extract_features(image)
        x = F.adaptive_avg_pool2d(x, 1).reshape(batch_size, -1)
        outputs = self.out(self.dropout(x))
        return outputs

And now we can build it:

In [ ]:
net = LeafModel(dls.c)

## Training

All that's left is to train our model.

We're not going to follow the freeze + unfreeze transfer learning methodology of fastai *only because I want this as close to a 1:1 as possible*. 

Let's build our `Learner`:

In [ ]:
learn = Learner(dls, net, loss_func=CrossEntropyLossFlat(),
               metrics=[accuracy]).to_native_fp16()

And now we'll train our model. To be equivalent we'll use `fit_flat_cos` with a `pct_start` of zero, along with `EarlyStopping`:

In [ ]:
learn.fit_flat_cos(10, 3e-4, pct_start=0.0,
                  cbs=[EarlyStoppingCallback(patience=3),
                      SaveModelCallback()])

We've achieved roughly 84-85% accuracy, so now we will export our model away and do inference with it in a seperate kernel [here](https://www.kaggle.com/muellerzr/fastai-abhishek-inference)

In [ ]:
learn.export('baseline')